# ライントレース+物体検知

## 物体検知側の下準備

In [1]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)  # カメラの起動

In [2]:
import darknet
import darknet_images
import time
"""
load model description and weights from config files
args:
    config_file (str): path to .cfg model file
    data_file (str): path to .data model file
    weights (str): path to weights
returns:
    network: trained model
    class_names
    class_colors
"""
network, class_names, class_colors = darknet.load_network(
    '/home/jetbot/g031r066/darknet/cfg/yolov4-tiny.cfg',  
    '/home/jetbot/g031r066/darknet/cfg/coco.data', 
    '/home/jetbot/g031r066/darknet/weights/yolov4-tiny.weights'
    )

"""
network, class_names, class_colors = darknet.load_network(
        '/home/jetbot/g031r066/darknet/custom/yolov4-tiny-custom.cfg', 
        '/home/jetbot/g031r066/darknet/custom/custom.data',  
        '/home/jetbot/g031r066/darknet/custom/backup/yolov4-tiny-custom_final.weights'
    )
"""

"\nnetwork, class_names, class_colors = darknet.load_network(\n        '/home/jetbot/g031r066/darknet/custom/yolov4-tiny-custom.cfg', \n        '/home/jetbot/g031r066/darknet/custom/custom.data',  \n        '/home/jetbot/g031r066/darknet/custom/backup/yolov4-tiny-custom_final.weights'\n    )\n"

## ライントレース側の下準備

In [3]:
# インポート
import torchvision
import torch

# 自分で作成したライントレースのモデルを読み込む
model_rf = torchvision.models.resnet18(pretrained=False)
model_rf.fc = torch.nn.Linear(512, 2)
model_rf.load_state_dict(torch.load('/home/jetbot/g031r066/note_nvidia/road_following/best_steering_models/[nvidia_circuit-ver1.2]_res18.pth'))

<All keys matched successfully>

In [4]:
# GPU側に転送
device = torch.device('cuda')
model_rf = model_rf.to(device)
model_rf = model_rf.eval().half()

In [5]:
# カメラから読み込む映像への前処理を実装
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [6]:
# カメラからの映像をセルの下に出力
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

# camera = Camera()
# image_widget = ipywidgets.Image()
# traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

# display(image_widget)

In [7]:
# 駆動系周りの準備
from jetbot import Robot

robot = Robot()  # 駆動系を制御できるモジュールのインポート

# 制御するバーの設定
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

# 描画
display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

# 自Jetbotにおいては、以下の設定だと良い感じに動く
# speed gain = 0.1 2022/09/09 0.15の方が綺麗
# steering gain = 0.05
# steering kd = 0
# steering gain = 0

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.05, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

In [8]:
# 現在の動作を示すバーを描画
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

## 本処理

In [9]:
from jetbot import bgr8_to_jpeg
import time
import queue

q = queue.Queue()  # メインスレッド<->サブスレッド で画像をやり取りするためのキューを作っておく

blocked_widget = ipywidgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
image_widget = ipywidgets.Image(format='jpeg', width=300, height=300)
label_widget = ipywidgets.IntText(value=1, description='tracked label')
speed_widget = ipywidgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = ipywidgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')
bbox_widget = ipywidgets.Image(format='jpeg', width=300, height=300)

display(ipywidgets.VBox([
    ipywidgets.HBox([image_widget, blocked_widget,bbox_widget]),
    label_widget,
    speed_widget,
    turn_gain_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

angle = 0.0
angle_last = 0.0

# ライントレースを実装する関数
def road_following(image):
    global angle, angle_last

    xy = model_rf(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)

    return 0

# 物体検知を実装する関数
def object_detection(snapshot):
    #snapshot = camera.value.copy()
    #images = darknet_images.load_images(snappshot)
    
    #image_name = images[0]
    prev_time = time.time()
    thresh = .30
    image, detections = darknet_images.image_detection(snapshot, network, class_names, class_colors, thresh)
    
    darknet.print_detections(detections, True)  # True: 各物体の座標・幅・高さを表示
    fps = int(1/(time.time() - prev_time))
    print("FPS: {}".format(fps))
    print("Predicted in {} seconds".format((time.time() - prev_time)))
    
    label_list=[]
    confidence_list=[]
    bbox_list=[]
    for label, confidence, bbox in detections: 
        label_list.append(label)
        confidence_list.append(confidence)
        bbox_list.append(bbox)

    print(label_list)   
    print(confidence_list)
    print(*bbox_list,sep='\n')
    
    q.put(image)  # メインスレッドにbboxを描画した画像を送る
    
    if("Traffic light" in label_list):
        # 信号機部分を切り出し
        i=1
        xmin, ymin, xmax, ymax = darknet.bbox2points(bbox_list[i])
        cropped_image = image[ymin+1:ymax-1, xmin+1:xmax-1]  # bboxの枠線が映らないようにちょっと狭めに切り取る
        
        #import numpy as np
        # 1.HSVに変換して白黒でマスク(2値化)
        hsv_cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)
        # 信号機の目:白,それ以外:黒 でマスクする 
        # 彩度(s)・明度(v)で絞り込む. 信号機の目は大体鮮やかなのでそのようになるように指定
        lower = np.array([0, 200,100])           # 抽出する色の下限(h,s,v)
        upper = np.array([150, 255 , 255])        # 抽出する色の上限(h,s,v)
        mask_traffic_light = cv2.inRange(hsv_cropped_image, lower, upper) # inRangeで元画像を２値化
        
        # 1の結果を基に、信号機の目部分の色はそのままに、それ以外を黒でマスク
        target = cv2.bitwise_and(hsv_cropped_image,hsv_cropped_image, mask=mask_traffic_light)
        
        #from ipywidgets import HBox
        """
        HBox([
            ipywidgets.Image(value=cv2.imencode(".jpg", cropped_image, (cv2.IMWRITE_JPEG_QUALITY, 100))[1].tobytes()),
            ipywidgets.Image(value=cv2.imencode(".jpg", hsv_cropped_image, (cv2.IMWRITE_JPEG_QUALITY, 100))[1].tobytes()),
            ipywidgets.Image(value=cv2.imencode(".jpg", mask_traffic_light, (cv2.IMWRITE_JPEG_QUALITY, 100))[1].tobytes()),
            ipywidgets.Image(value=cv2.imencode(".jpg", target, (cv2.IMWRITE_JPEG_QUALITY, 100))[1].tobytes())
        ])
        """
        # 信号機の目以外は黒色(H=0)でマスクしている為、Hの平均値を出しても小さい値にしかならないので意味がない
        # → なのでHのminとmaxを出す(min要らないけど)
        h_min = target.T[0].flatten().min()
        h_max = target.T[0].flatten().max()
        print("h_min: %s" % (str(h_min)))
        print("h_max: %s" % (str(h_max)))
        
        # Hの最大値で判別する
        if(0 <= h_max <= 30):
            robot.stop() # 赤信号なので一時停止
            time.sleep(0.1) 
            print("Traffic_light : red")
        elif(30<h_max and h_max<=150):  # 緑と青は取り敢えず一緒にした
            print("Traffic_light : green")
        else:
            print("Traffic_light : none")
            
    return 0

In [10]:
import threading
from IPython.display import clear_output
def execute(change):
    # start_time = time.time()
    
    snapshot = change['new']  # 映像読み込み type(snapshot) = numpy.ndarray

    # ライントレースと物体検知のスレッドをそれぞれ作成
    thread_rf = threading.Thread(target = road_following, args=(snapshot,))
    thread_od = threading.Thread(target = object_detection, args=(snapshot,))

    # 各スレッド実行開始
    thread_rf.start()
    thread_od.start()

    # 各スレッド内の処理が終わるまで待つ
    thread_rf.join()
    thread_od.join()

    # 出力されているカメラ映像の更新
    #image_widget.value = bgr8_to_jpeg(snapshot)
    image_widget.value = cv2.imencode(".jpg", snapshot, (cv2.IMWRITE_JPEG_QUALITY, 100))[1].tobytes()
    bbox_widget.value  = cv2.imencode(".jpg", q.get(), (cv2.IMWRITE_JPEG_QUALITY, 100))[1].tobytes()
    
    # finish_time = time.time()
    # excute_time = finish_time - start_time
    # print(excute_time)
   # clear_output(True)
    
execute({'new': camera.value})
camera.observe(execute, names='value')


Objects:
bottle: 31.85%    (left_x: 208   top_y:  120   width:   110   height:  231)
bottle: 53.28%    (left_x: 210   top_y:  211   width:   104   height:  256)
FPS: 1
Predicted in 0.5196638107299805 seconds
['bottle', 'bottle']
['31.85', '53.28']
(208.0537567138672, 119.6723861694336, 109.71839904785156, 231.34703063964844)
(209.5205078125, 211.4897003173828, 103.58914184570312, 255.91751098632812)

Objects:
bottle: 30.14%    (left_x: 209   top_y:  119   width:   107   height:  236)
cup: 32.73%    (left_x: 209   top_y:  214   width:   106   height:  253)
chair: 35.14%    (left_x: 371   top_y:  246   width:   95   height:  185)
bottle: 53.49%    (left_x: 209   top_y:  214   width:   106   height:  253)
FPS: 8
Predicted in 0.11802220344543457 seconds
['bottle', 'cup', 'chair', 'bottle']
['30.14', '32.73', '35.14', '53.49']
(208.7075958251953, 118.74530792236328, 107.13407135009766, 235.86862182617188)
(209.14776611328125, 213.61883544921875, 105.54763793945312, 253.09645080566406)
(370

Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing.

We accomplish that with the observe function.

>WARNING: This code will move the robot!! Please make sure your robot has clearance and it is on Lego or Track you have collected data on. The road follower should work, but the neural network is only as good as the data it's trained on!

In [11]:
camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
camera.stop()
robot.stop()


chair: 33.2%    (left_x: 361   top_y:  234   width:   101   height:  178)
bottle: 50.02%    (left_x: 244   top_y:  232   width:   120   height:  260)
cup: 56.66%    (left_x: 246   top_y:  208   width:   122   height:  279)
FPS: 5
Predicted in 0.17640423774719238 seconds
['chair', 'bottle', 'cup']
['33.2', '50.02', '56.66']
(361.16925048828125, 234.1702423095703, 100.9041748046875, 177.65853881835938)
(244.392578125, 231.5783233642578, 119.94430541992188, 259.6139831542969)
(246.1241912841797, 207.84762573242188, 121.73562622070312, 278.83740234375)

Objects:
cup: 51.28%    (left_x: 246   top_y:  212   width:   126   height:  277)
bottle: 66.11%    (left_x: 246   top_y:  212   width:   126   height:  277)
FPS: 6
Predicted in 0.15590834617614746 seconds
['cup', 'bottle']
['51.28', '66.11']
(246.2762451171875, 211.97479248046875, 125.6891098022461, 276.9694519042969)
(246.2762451171875, 211.97479248046875, 125.6891098022461, 276.9694519042969)


In [23]:
camera.stop()
robot.stop()